# Multi-class Bayesian Logistic Regression with PyMC3 ~ Softmax Regression
* From the "Bayesian Analysis with Python" book repository
* written by Osvaldo Martin 
- credit: https://github.com/aloctavodia/BAP/blob/master/code_3_11/Chp4/04_Generalizing_linear_models.ipynb

In [1]:
import time
import pymc3 as pm
import numpy as np
import pandas as pd
import theano.tensor as tt
import seaborn as sns
import scipy.stats as stats
from scipy.special import expit as logistic
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [2]:
# import arviz as az
# az.style.use('arviz-darkgrid')

## Explore one classification task of MiniImagenet dataset

In [3]:
# explore the full data set (i.e. support + augmented)
# miniImagenet_train = pd.read_csv('./data/miniImagenet/X_aug_run_0.csv')

# try the pca reduced full dataset
miniImagenet_train = pd.read_csv('./data/miniImagenet/miniImagenet_pca/X_aug_pca_run_0.csv')
miniImagenet_train.head()

,f#1,f#2,f#3,f#4,f#5,label
0,-0.345973,-0.141867,-0.046007,-0.169700,0.251301,0.0
1,-0.221673,-0.003457,0.071326,0.349216,0.067107,1.0
2,-0.000117,-0.071104,-0.286720,-0.062245,-0.183093,2.0
3,-0.308013,0.377608,0.151592,-0.028319,-0.055166,3.0
4,-0.461263,-0.318390,0.377740,-0.108410,-0.004293,4.0


In [4]:
num_classes = len(miniImagenet_train.label.unique())
num_classes

5

## Preparing the data for Bayesian Softmax Regression

In [5]:
NUM_SELECTED_POINTS = 100

In [6]:
y_train = pd.Categorical(miniImagenet_train['label'][:NUM_SELECTED_POINTS]).codes       # {0,1,2,3,4} 
x_train_cols = miniImagenet_train.columns.difference(['label'])

x_train = miniImagenet_train[x_train_cols].values[:NUM_SELECTED_POINTS]                # shape: (number of datapoints, number of features)
num_data_points, num_feats = x_train.shape
print(f"number of data points: {num_data_points}")

number of data points: 100


In [7]:
num_classes

5

### Standardizing the features values (if needed)

In [8]:
# std_scaler = StandardScaler()
# std_scaler.fit(x_train)

In [9]:
# sum(x_train.mean(axis=0) == std_scaler.mean_)

In [10]:
# sum(x_train.std(axis=0) == std_scaler.scale_)

## Softmax Bayesian Regression

In [11]:
start = time.time()

num_samples = 500
num_chains = 2


with pm.Model() as model_s:
    α = pm.Normal('α', mu=0, sd=5, shape=num_classes)             # bias term
    β = pm.Normal('β', mu=0, sd=5, shape=(num_feats,num_classes))         # feature weight
    μ = pm.Deterministic('μ', α + pm.math.dot(x_train, β))  # linear combination of the features
    θ = tt.nnet.softmax(μ)                              # softmax prediction (150, 3)
    yl = pm.Categorical('yl', p=θ, observed=y_train)
    # idata_s = pm.sample(num_samples, target_accept=0.9, return_inferencedata=True)
    idata_s = pm.sample(num_samples, chains=num_chains,  target_accept=0.9, return_inferencedata=True)
    
end = time.time()
print(f"elapsed time for {num_data_points} data points and {num_feats} features is: {(end - start):.4f} seconds")

/home/sara/anaconda3/envs/bayes/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
/home/sara/anaconda3/envs/bayes/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
/home/sara/anaconda3/envs/bayes/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: 

elapsed time for 100 data points and 5 features is: 15.1587 seconds


In [12]:
idata_s[0]['μ'].shape
idata_s[0].keys()

dict_keys(['α', 'β', 'μ'])

In [13]:
len(idata_s)

500

In [14]:
idata_s.nchains

2

In [15]:
idata_s.varnames

['α', 'β', 'μ']

In [16]:
idata_s.chains

[0, 1]

In [17]:
# DONT REMOVE
# i = 0
# for pmc_iter in idata_s.points(): # 8000 iterations (i.e. number of points = 4 chains x 2000 iterations)
#     logits_vals = np.vstack(
    # print(pmc_iter.keys())        # dict_keys(['α', 'β', 'μ'])
    # print(pmc_iter['μ'].shape)    # (150,3) for each iteration we have (150,3) μ : logit values of 150 datapoints: (150, 3) (to be passed to softmax predcition next)
    # print(pmc_iter['β'].shape)    # (4,3)   for each iteration we have   (4,3) β
    # print(pmc_iter['α'].shape)    # (3,)    for each iteration we have    (3,) α

#     i+=1
# print(i)

## Accuracy for training data

In [18]:
# DONT REMOVE
print(idata_s.get_values('μ').shape)   # shape (8000, 150, 3) ---> (num_iters, num_data_points, num_classes)
print(idata_s.get_values('α').shape)   # shape (8000, 3)      ---> (num_iters, num_classes)
print(idata_s.get_values('β').shape)   # shape (8000, 4, 3)   ---> (num_iters, num_feats, num_classes)

(1000, 30, 5)
(1000, 5)
(1000, 5, 5)


In [19]:
# get logits by averaging over the num_iters (here 8000)
# shape of μ is (num_iters, num_data_points, num_classes)
data_pred = idata_s.get_values('μ').mean(axis=0)   # logits for training data: shape (150, 3)

# softmax prediction for the 150 datapoint in the training dataset
y_pred = [np.exp(point)/np.sum(np.exp(point), axis=0) for point in data_pred]   # len(y_pred) = num_data_points

# accuracy of the training data = 98%
print(f'{np.sum(y_train == np.argmax(y_pred, axis=1)) / len(y_train):.2f}')  

0.93


## Predicting test data

In [20]:
def get_softmax_prediction(x_test, y_test, α, β, estimate_type='mean'):   
    '''
    x_test: held-out test data.     shape = (num_data_points, num_feats)
    y_test: held-out true labels.   shape = (num_data_points,)
    α: bias term.                   shape = (num_iters, num_classes)
    β: features weights.            shape = (num_iters, num_feats, num_classes)
    estimate_type: available options = ['mean', 'median', 'mode'], default value is: 'mean'
    '''
    if estimate_type == "mean":
        y_pred = α.mean(axis=0) + np.dot(x_test, β.mean(axis=0))    # first term shape: ?? , second term shape: (num_points, num_classes)
    
    elif estimate_type == "median":
        y_pred = np.median(α, axis=0) + np.dot(x_test, np.median(β, axis=0))
        
    elif estimate_type == "mode":
        α_mode = stats.mode(α, axis=0)[0].squeeze()
        β_mode = stats.mode(β, axis=0)[0].squeeze()
        y_pred = α_mode + np.dot(x_test, β_mode)
    proba = np.exp(y_pred).T/np.sum(np.exp(y_pred), axis=1)
    p_class = np.argmax(proba, axis=0)
    
    accuracy = np.sum(y_test == np.argmax(y_pred, axis=1)) / len(y_test)
    return proba, p_class, accuracy * 100

### Predict support set (i.e. training data)

In [21]:
# Bayesian parameters from the drawn samples
α = idata_s.get_values('α')
β = idata_s.get_values('β')

In [22]:
# hint: 'mode' estimate type takes more time than 'mean' and 'median'
y_proba, y_pred, accuracy = get_softmax_prediction(x_train, y_train, α, β, estimate_type='median')
accuracy

93.33333333333333

### Predict query set (held-out data set)

In [23]:
# miniImagenet_test = pd.read_csv('./data/miniImagenet/query_run_0.csv')
miniImagenet_test = pd.read_csv('./data/miniImagenet/miniImagenet_pca/query+full_pca_run_0.csv')
miniImagenet_test.shape

(75, 6)

In [24]:
y_test = pd.Categorical(miniImagenet_test['label']).codes       # {0,1,2,3,4} 
x_test_cols = miniImagenet_test.columns.difference(['label'])
x_test = miniImagenet_test[x_test_cols].values                 # shape: (number of datapoints, number of features)

In [25]:
y_test_proba, y_test_pred, test_accuracy = get_softmax_prediction(x_test, y_test, α, β, estimate_type='median')
test_accuracy

57.333333333333336